# Imports

In [1]:
import pandas as pd
import numpy as np
from pyvis import network as pvnet
import networkx as nx
from networkx import MultiDiGraph
from enum import Enum, auto
from datetime import datetime
from typing import Self, Optional, Union, Tuple

# Raw to Standardized Form Conversion Code (DO NOT EDIT !!!)

## Create an Enum for Entities

In [2]:
class Ent(Enum):

    ENTITY      = 0
    LOCATION    = auto()
    TRANSPORTER = auto()
    SUPPLIER    = auto()
    MATERIAL    = auto()
    PRODUCER    = auto()
    PRODUCT     = auto()
    DISTRIBUTOR = auto()
    CUSTOMER    = auto()


## The Entity Database

- A comman database for storing all entities while graph creation using `numpy.array` in a dict

- Each entity will be stored in unique array, with its name as key

In [3]:
class EntityDatabase:

    # Initializer
    def __init__(
        self,
        db_path:str = 'dataset',  # Path to store the database
        db_name:str = datetime.now().strftime("%Y-%m-%d %H-%M-%S")  # Filename to save
    ):

        # Name
        self.name = 'Entity Database'

        # Store path and name in variables
        self.db_path = db_path
        self.db_name = db_name

        # Create empty dict
        self.storage = {}

        # Populate according to the Entities in `Ent`
        for entity in Ent:
            self.storage[entity.name] = np.empty((0,), dtype=object)

    # Add entity to the storage
    # TODO: Check for Duplication
    def add(self, entity: 'Entity'):
        type = entity.type

        self.storage[type.name] = np.append(self.storage[type.name], entity)
    
    # Print the complete Entity Database
    def show(self) -> Self:
        print(f'{self.name}')

        cnt = 1
        for entity_name in self.storage:
            for entity in self.storage[entity_name]:
                print(f'\t[{cnt:3}] {entity}')
                cnt += 1
        return self
    
    def __str__(self):
        return f"<Class '{self.__class__.__name__}'|{self.name}>"

    # Reset
    def reset(self):
        self.__init__()


In [4]:
# Create the DB
DB = EntityDatabase()

## Entity (Base Class)

In [5]:
class Entity:

    # Initializer
    def __init__(self, name: Optional[str] = None):

        # Name of the Entity
        self.name = name

        # Type of Entity
        self.type = Ent.ENTITY

        # Properties
        self._price = None
        self._time = None
        self._capacity = None
        self._location = None
        self._supplier = None
        self._extra_properties = dict()
    
    # String Output
    def __str__(self):
        return f"<Class '{self.__class__.__name__}'|{self._name}>"
    
    # The price of entity (per item)
    def price(self, value: float) -> Self:
        self._price = value
        return self

    # The time required (in Hours)
    def time(self, time_required: int) -> Self:
        self._time = time_required
        return self
    
    # Set the capacity of the Entity
    def capacity(self, capa: float) -> Self:
        self._capacity = capa
        return self
    
    # Set the location of the Entity
    def at(self, loc: 'Location') -> Self:
        self._location = loc
        return self
    
    # Entity Supplied by
    def by(self, supplied_by: 'Supplier') -> Self:
        self._supplier += (supplied_by, )
        return self
    
    # Extra Porps
    def extra_properties(self, properties: dict) -> Self:

        self._extra_properties = properties
        return self

    # Get Node: Return all the info to generate a node in the graph
    def get_Node(self) -> dict:

        node_data = {}

        if self._extra_properties:
            for prop, value in self._extra_properties.items():
                node_data[prop] = value
        
        return node_data
    
    # Get NodeLink: Returns the edge info between two nodes, this entity connects
    def get_NodeLink(self) -> dict:
        raise NotImplementedError


## Location

In [6]:
class Location(Entity):

    # Initializer
    def __init__(self, name: Optional[str]):
        super().__init__(name)

        # Type of Entity
        self.type = Ent.LOCATION

        # Default Name:
        if name is None:
            n = len(DB.storage[self.type.name])
            self.name = f"l{n}"

        # Add this to DB
        DB.add(self)
    
    def __str__(self) -> str:

        return f'{self.type.name:10}  {self.name}'


## Transporter

In [7]:
class Transporter(Entity):

    # Initializer
    def __init__(
        self,
        from_location: Location,
        to_location: Location,
        name: Optional[str] = None,
        rate: Optional[float] = None,
        required_time: Optional[int] = None,
        cap: Optional[float] = None
    ):
        super().__init__(name)

        # Type of Entity
        self.type = Ent.TRANSPORTER

        # Default Name:
        if name is None:
            n = len(DB.storage[self.type.name])
            self.name = f"t{n}"

        # Properties
        self._price          = rate
        self._time           = required_time  # In Hours
        self._capacity       = cap
        self._from           = from_location
        self._to             = to_location

        # Add this to DB
        DB.add(self)

    # Get NodeLink Info
    def get_NodeLink(self) -> dict:
        
        # A dict with all properties
        link_data = {
            'fromLoc': self._from.name,
            'toLoc': self._to.name
        }

        if self._price:
            link_data['price'] = self._price
        
        if self._time:
            link_data['time'] = self._time
        
        if self._capacity:
            link_data['capacity'] = self._capacity
        
        return link_data

    # Show the Entity
    def __str__(self):

        info = f'{self.type.name:10}  {self.name} | {self._from.name} ==> {self._to.name}'

        if self._price:
            info += f' | {self._price}/per unit'
        
        if self._time:
            info += f' | {self._time} hrs'
        
        if self._capacity:
            info += f' | {self._capacity} units capacity'
        
        info += " |"

        return info


## Supplier

In [8]:
class Supplier(Entity):
    
    # Initializer
    def __init__(
        self,
        name: Optional[str] = None,
        loc: Optional[Location] = None,
        varPrice: Optional[float] = None,
        required_time: Optional[int] = None,
        cap: Optional[float] = None
    ):
        super().__init__(name)

        # Type of Entity
        self.type = Ent.SUPPLIER

        # Default Name:
        if name is None:
            n = len(DB.storage[self.type.name])
            self.name = f"s{n}"

        # Properties
        self._price          = varPrice
        self._time           = required_time  # In Hours
        self._capacity       = cap
        self._location       = loc
        self._supplied_entities = tuple()

        # Add this to DB
        DB.add(self)

    # Add the supplied Entities
    def supplies(self, supplied_entity: Union['Material', 'Product', tuple]) -> Self:
        
        if not isinstance(supplied_entity, tuple):
            supplied_entity = (supplied_entity, )
        
        self._supplied_entities += supplied_entity

        # Remove Duplicates
        self._supplied_entities = tuple(set(self._supplied_entities))

        # Add this to product / Material as well
        for entity in supplied_entity:
            if self not in entity._supplier:
                entity.by(self)

        return self

    # Print detials
    def __str__(self) -> str:
        info = f'{self.type.name:10}  {self.name}'

        if self._location:
            info += f' | {self._location.name}'
        
        if self._price:
            info += f' | {self._price}/unit'
        
        if self._time:
            info += f' | {self._time} hrs'
        
        if self._capacity:
            info += f' | {self._capacity} units capacity'
        
        if self._supplied_entities:
            info += f' | Entities Supplied: {", ".join([entity.name for entity in self._supplied_entities])}'

        info += " |"
        return info

    # Get Node Info
    def get_Node(self) -> dict:
        
        # A dict with all properties
        node_data = super().get_Node()

        if self._price:
            node_data['vCost'] = self._price
        
        if self._time:
            node_data['time'] = self._time
        
        if self._capacity:
            node_data['capacity'] = self._capacity
        
        return node_data

    # Get Node Info
    def get_NodeLink(self) -> dict:

        link_data = {}
        
        # A dict with all properties
        if self._supplied_entities:
            link_data['supplies'] = self._supplied_entities

        return link_data


## Material

In [9]:
class Material(Entity):

    # Initializer
    def __init__(
        self,
        name: Optional[str] = None,
        price: Optional[float] = None,
        supplied_by: Optional[Supplier] = tuple()
    ):
        super().__init__(name)

        # Type of Entity
        self.type = Ent.MATERIAL

        # Default Name:
        if name is None:
            n = len(DB.storage[self.type.name])
            self.name = f"m{n}"

        # Add Properties
        self._price = price

        if not isinstance(supplied_by, tuple):
            supplied_by = (supplied_by, )
        self._supplier = supplied_by

        # Required for
        self._required_for = tuple()
        self._produced_by = tuple()

        # Add this to DB
        DB.add(self)
    
    # Show detials
    def __str__(self) -> str:
        info = f'{self.type.name:10}  {self.name}'

        if self._location:
            info += f' | {self._location.name}'
        
        if self._price:
            info += f' | {self._price}/unit'
        
        if self._time:
            info += f' | {self._time} hrs'
        
        if self._capacity:
            info += f' | {self._capacity} units capacity'
        
        if self._supplier:
            info += f' | Supplied by: {", ".join([entity.name for entity in self._supplier])}'
        
        if self._required_for:
            info += f' | Required for: {", ".join([entity.name for entity in self._required_for])}'

        if self._extra_properties:
            info += f' | {" | ".join([str(prop)+": "+str(value) for prop, value in self._extra_properties.items()])}'

        if self._produced_by:
            info += f' | Produced by: {", ".join([entity.name for entity in self._produced_by])}'

        info += " |"
        return info

    # Add Supplier
    def by(self, suppliers: Union[Supplier, 'Producer', tuple]) -> Self:

        if not isinstance(self._supplier, tuple):
            self._supplier = tuple()
        
        if not isinstance(suppliers, tuple):
            suppliers = (suppliers, )

        # Distinguish between Producer and Supplier
        for entity in suppliers:

            if entity.type == Ent.PRODUCER:
                self._produced_by += (entity, )
            
            elif entity.type in (Ent.SUPPLIER, Ent.DISTRIBUTOR):
                self._supplier += (entity, )

        # Remove Duplicates
        self._supplier = tuple(set(self._supplier))
        self._produced_by = tuple(set(self._produced_by))

        # Add to the Suppliers
        for entity in suppliers:

            if entity.type == Ent.SUPPLIER and self not in entity._supplied_entities:
                entity.supplies(self)
            
            elif entity.type == Ent.PRODUCER and self not in entity._products:
                entity.produces(self)

        return self

    # Add product
    def required_for(self, products: Union['Product', Tuple['Product', ...]]) -> Self:

        if not isinstance(products, tuple):
            products = (products, )
        
        self._required_for += products

        # Remove Duplicates
        self._required_for = tuple(set(self._required_for))

        # Add to the Products
        for product in products:
            if self not in product.materials_required:
                product.ingredient(self)
        
        return self

    # Get Node Info
    def get_Node(self) -> dict:
        
        # A dict with all properties
        node_data = super().get_Node()

        if self._price:
            node_data['vCost'] = self._price

        return node_data

    # Get Node Info
    def get_NodeLink(self) -> dict:
        
        # A dict with all properties
        link_data = {}

        if self._supplier:
            link_data['supplied_by'] = self._supplier

        if self._required_for:
            link_data['required_for'] = self._required_for

        if self._produced_by:
            link_data['produced_by'] = self._produced_by

        return link_data


## Product

In [10]:
class PriceNotProviced(Exception):
    pass

In [11]:
class Product(Entity):

    # Initializer
    def __init__(
        self,
        name: Optional[str] = None,
        price: Optional[float] = None,
    ):
        super().__init__(name)

        # Type of Entity
        self.type = Ent.PRODUCT

        # Default Name:
        if name is None:
            n = len(DB.storage[self.type.name])
            self.name = f"p{n}"

        # Add Properties
        self._price = price
        self._supplier = tuple()
        self.materials_required = tuple()
        self._produced_by = tuple()

        # Add this to DB
        DB.add(self)

    # Add required Ingrdients
    def ingredient(self, materials_required: Union[Material, Tuple[Material, ...]]) -> Self:

        if not isinstance(self.materials_required, tuple):
            self.materials_required = tuple()
        
        elif not isinstance(materials_required, tuple):
            materials_required = (materials_required,)
        
        self.materials_required += materials_required

        # Remove Duplicates
        self.materials_required = tuple(set(self.materials_required))

        # Add to the Materials
        for material in materials_required:
            material: Material
            if self not in material._required_for:
                material.required_for(self)

        return self
    
    # Add the Producer of Supplier for the product
    def by(self, supplied_by: Union['Distributor', 'Supplier', 'Producer', Tuple]) -> Self:

        if not isinstance(self._supplier, tuple):
            self._supplier = tuple()
        
        if not isinstance(supplied_by, tuple):
            supplied_by = (supplied_by, )

        # Distinguish between Producer and Supplier
        for entity in supplied_by:

            if entity.type == Ent.PRODUCER:
                self._produced_by += (entity, )
            
            elif entity.type in (Ent.SUPPLIER, Ent.DISTRIBUTOR):
                self._supplier += (entity, )

        # Remove Duplicates
        self._supplier = tuple(set(self._supplier))
        self._produced_by = tuple(set(self._produced_by))

        # Add to the Suppliers
        for entity in supplied_by:

            if entity.type == Ent.SUPPLIER and self not in entity._supplied_entities:
                entity.supplies(self)
            
            elif entity.type == Ent.PRODUCER and self not in entity._products:
                entity.produces(self)

        return self

    # Show detials
    def __str__(self) -> str:
        info = f'{self.type.name:10}  {self.name}'
        
        if self._price:
            info += f' | {self._price}/unit'
        
        if self._time:
            info += f' | {self._time} hrs'
        
        if self._capacity:
            info += f' | {self._capacity} units capacity'
        
        if self.materials_required:
            info += f' | Ingredients: {", ".join([m.name for m in self.materials_required])}'
        
        if self._supplier:
            info += f' | Suppliers: {", ".join([supplier.name for supplier in self._supplier])}'
        
        if self._produced_by:
            info += f' | Producers: {", ".join([producer.name for producer in self._produced_by])}'


        info += " |"
        return info

    # Get Node Info
    def get_Node(self) -> dict:
        
        # A dict with all properties
        node_data = super().get_Node()

        if self._price:
            node_data['vCost'] = self._price
        
        return node_data

    # Get Node Info
    def get_NodeLink(self) -> dict:
        
        # A dict with all properties
        link_data = {}

        if self._supplier:
            link_data['supplied_by'] = self._supplier
        
        if self._produced_by:
            link_data['produced_by'] = set(self._produced_by)
        
        if self.materials_required:
            link_data['materials_required'] = [mat for mat in self.materials_required]

        return link_data


## Producer / Manufacturer

In [12]:
class Producer(Entity):
    
    # Initializer
    def __init__(
        self,
        name: Optional[str] = None,
        varCost: Optional[float] = None,
        capa: Optional[int] = None
    ):
        super().__init__(name)

        # Type of Entity
        self.type = Ent.PRODUCER

        # Default Name:
        if name is None:
            n = len(DB.storage[self.type.name])
            self.name = f"P{n}"

        # Add Properties
        self._varCost = varCost
        self._location = None
        self._capacity = capa
        self._supplies_to = tuple()
        self._products = tuple()

        # Add this to DB
        DB.add(self)

    # Set the VarCost
    def varCost(self, varcost: float) -> Self:
        self._varCost = varcost
        return self

    # Show detials
    def __str__(self) -> str:
        info = f'{self.type.name:10}  {self.name}'

        if self._location:
            info += f' | {self._location.name}'
        
        if self._price:
            info += f' | {self._price}/unit'
        
        if self._varCost:
            info += f' | {self._varCost}/unit'
        
        if self._time:
            info += f' | {self._time} hrs'
        
        if self._capacity:
            info += f' | {self._capacity} units capacity'

        if self._supplies_to:
                info += f' | Supplies to: {", ".join([s.name for s in self._supplies_to])}'
        
        if self._products:
            info += f' | Produces: {", ".join([entity.name for entity in self._products])}'

        info += " |"

        return info

    # Add products produced
    def produces(self, products: Union[Product, Tuple[Product, ...]]) -> Self:

        # Convert to tuple
        if not isinstance(products, tuple):
            products = (products, )

        # Add
        self._products += products

        # Remove Duplicates
        self._products = tuple(set(self._products))

        # Add to the Products
        for product in products:
            if self not in product._produced_by:
                print(product)
                product.by(self)

        return self

    # Add the Supplying Destinations
    def supplies(self, supplies_to: 'Distributor') -> Self:
        
        # Check if Tuple
        if not isinstance(supplies_to, tuple):
            supplies_to = (supplies_to,)
        
        # Add
        self._supplies_to += supplies_to

        # Remove Duplicates
        self._supplies_to = tuple(set(self._supplies_to))

        # Add the Distributors
        for dest in supplies_to:
            dest: Distributor
            if self not in dest._suppliers:
                dest.supplier(self)

        return self

    # Get Node Info
    def get_Node(self) -> dict:
        
        # A dict with all properties
        node_data = super().get_Node()

        if self._price:
            node_data['vCost'] = self._price
        
        if self._time:
            node_data['time'] = self._time
        
        if self._capacity:
            node_data['capacity'] = self._capacity
        
        return node_data

    # Get Node Info
    def get_NodeLink(self) -> dict:
        
        # A dict with all properties
        link_data = {}

        if self._supplies_to:
            link_data['supplies_to'] = self._supplies_to
        
        if self._products:
            link_data['produces'] = self._products

        return link_data


## The Distributors

In [13]:
class Distributor(Producer):

    # Initializer
    def __init__(
        self,
        name: Optional[str] = None,
        varCost: Optional[float] = None,
        capa: Optional[int] = None
    ):
        super().__init__(name)

        # Type of Entity
        self.type = Ent.DISTRIBUTOR

        # Default Name:
        if name is None:
            n = len(DB.storage[self.type.name])
            self.name = f"d{n}"

        # Add Properties
        self._varCost = varCost
        self._location = None
        self._capacity = capa
        self._suppliers = tuple()
        self._inventory = tuple()

        # Add this to DB
        DB.add(self)
    
    # Show detials
    def __str__(self) -> str:
        info = f'{self.type.name:10}  {self.name}'

        if self._location:
            info += f' | {self._location.name}'
        
        if self._price:
            info += f' | {self._price}/unit'
        
        if self._time:
            info += f' | {self._time} hrs'
        
        if self._capacity:
            info += f' | {self._capacity} units capacity'
        
        if self._suppliers:
            info += f' | Suppliers: {", ".join([entity.name for entity in self._suppliers])}'

        if self._inventory:
            info += f' | Inventory: {", ".join([entity.name for entity in self._inventory])}'

        info += " |"
        return info

    # Add Suppliers
    def supplier(self, suppliers: Union[Producer, Tuple[Producer, ...]]) -> Self:
        
        # Check if not tuple
        if not isinstance(suppliers, tuple):
            suppliers = (suppliers, )
        
        # Add
        self._suppliers += suppliers

        # Remove Duplicates
        self._suppliers = tuple(set(self._suppliers))

        # Add to Producers
        for supplier in suppliers:
            if self not in supplier._supplies_to:
                supplier.supplies(self)
            
            # Add to Products
            for product in supplier._products:

                product : Product

                # Add to Inventory
                if product not in self._products:
                    self._inventory += (product, )

                if self not in product._supplier:
                    product.by(self)
        
        return self

    # Node Link
    def get_NodeLink(self) -> dict:
        link_data =  super().get_NodeLink()

        if self._suppliers:
            link_data['supplier'] = self._suppliers
        
        if self._inventory:
            link_data['supplies'] = self._inventory
        
        return link_data


## Customer

In [14]:
class Customer(Entity):

    # Initializer
    def __init__(
        self,
        name: Optional[str] = None,
        product: Optional[Product] = None,
        demand: Optional[int] = None,
        time: Optional[int] = None
    ):
        super().__init__(name)

        # Type of Entity
        self.type = Ent.CUSTOMER

        # Default Name:
        if name is None:
            n = len(DB.storage[self.type.name])
            self.name = f"c{n}"

        # Add Properties
        self._demand = demand
        self._location = None
        self._time = time
        self._product = product

        # Add this to DB
        DB.add(self)
    
    # Show detials
    def __str__(self) -> str:
        info = f'{self.type.name:10}  {self.name}'

        if self._location:
            info += f' | {self._location.name}'
        
        if self._product:
            info += f' | Demands: {self._product.name}'
        
        if self._demand:
            info += f' | {self._demand} units'
        
        if self._time:
            info += f' | {self._time} hrs'
        
        info += " |"
        return info

     # Get Node Info
    def get_Node(self) -> dict:
        
        # A dict with all properties
        node_data = super().get_Node()

        if self._demand:
            node_data['demand'] = self._demand
        
        return node_data

    def demand(self, demand: float) -> Self:
        self._demand = demand
        return self

    def product(self, prod: Product) -> Self:
        self._product = prod
        return self
    
    # Get Node Info
    def get_NodeLink(self) -> dict:
        
        # A dict with all properties
        link_data = {
            'demands': self._product
        }
        return link_data


## Supply Chain Network Class

In [15]:
class SupplyNetwork:

    def __init__(self, name: Optional[str] = 'Supply Network'):

        # Set the Name
        self.name = name

        # The main Database
        self.db = None

        # Set different graphs to none
        self.transportation_graph = None
        self.supply_flow_graph    = None
        self.knowledge_graph      = None
    
    # Add the Supply Chain DB
    def add_DB(self, db: EntityDatabase) -> None:
        self.db = db
    
    # Create a Transportation Graph
    def create_transportation_graph(self, as_label:bool = True) -> Tuple[MultiDiGraph, dict]:

        # Loop through all transportation nodes
        transportation_data = self.db.storage[Ent.TRANSPORTER.name]

        graph_dict = {}
        for transporter in transportation_data:
            transporter: Transporter
            link_data = transporter.get_NodeLink()

            # Get from and to Node
            fromNode = link_data.pop('fromLoc')
            toNode   = link_data.pop('toLoc')

            # Check if fromNode is present in the graph_data
            if not fromNode in graph_dict:
                graph_dict[fromNode] = {}
            
            # Check if to is present in the fromNode dict
            if not toNode in graph_dict[fromNode]:
                graph_dict[fromNode][toNode] = {}
            
            # Get the current number of links between nodes
            n = len(graph_dict[fromNode][toNode])

            # Create Label (or title)
            label = f"{transporter.name}\n"
            for key, value in link_data.items():
                label += f"{key}:{value}\n"
            
            # Add label (or title) to link data
            if as_label:
                link_data['label'] = label[:-1]
            else:
                link_data['title'] = label[:-1]
            
            # Add to Graph dict
            graph_dict[fromNode][toNode][n] = link_data

        self.transportation_graph = MultiDiGraph(graph_dict)
        return self.transportation_graph, graph_dict

    # Create supply Flow
    def create_supply_flow(self, as_label:bool = True) -> MultiDiGraph: 

        # Get all players
        supplier_data       = self.db.storage[Ent.SUPPLIER.name]
        producer_data       = self.db.storage[Ent.PRODUCER.name]
        distributor_data    = self.db.storage[Ent.DISTRIBUTOR.name]
        customer_data       = self.db.storage[Ent.CUSTOMER.name]
        material_data       = self.db.storage[Ent.MATERIAL.name]
        product_data        = self.db.storage[Ent.PRODUCT.name]

        # Supply Flow Graph
        self.supply_flow_graph = MultiDiGraph()

        # Create Nodes
        for entity_type in [supplier_data, material_data, producer_data, product_data, distributor_data, customer_data]:
            for entity in entity_type:

                entity: Supplier | Producer | Distributor | Customer | Material | Product

                # Get Node details
                node_data = entity.get_Node()

                # Create Label
                label = f"{entity.name}\n"
                for key, value in node_data.items():
                    label += f"{key}:{value}\n"
                
                # Add label (or title) to link data
                if as_label:
                    node_data['label'] = label[:-1]
                else:
                    node_data['title'] = label[:-1]

                # Add Node
                self.supply_flow_graph.add_nodes_from([(entity.name, node_data)])
        

        # Create NodeLinks
        for entity_type in [material_data, product_data, customer_data, producer_data, supplier_data]:
            for entity in entity_type:

                entity: Material | Product | Customer | Supplier

                # Get NodeLink Details
                link_data = entity.get_NodeLink()
                
                for key, value in link_data.items():

                    defined_keys = ['supplied_by', 'materials_required', 'supplies_to', 'supplies', 'produces', 'demands', 'required_for', 'supplier', 'produced_by']

                    if key in defined_keys:
                        label = key
                    else:
                        label = 'not_defined'

                    # Add label (or title) to link data
                    info = {}
                    if as_label:
                        info['label'] = label
                    else:
                        info['title'] = label

                    try:
                        for ent in value:
                            self.supply_flow_graph.add_edges_from([(entity.name, ent.name, info)])
                    except TypeError:
                        self.supply_flow_graph.add_edges_from([(entity.name, value.name, info)])
        

        return self.supply_flow_graph
                
    # Knowledge Graph
    def create_knowledge_graph(self, as_label:bool = True) -> MultiDiGraph:

        # Get all players
        supplier_data       = self.db.storage[Ent.SUPPLIER.name]
        producer_data       = self.db.storage[Ent.PRODUCER.name]
        distributor_data    = self.db.storage[Ent.DISTRIBUTOR.name]
        customer_data       = self.db.storage[Ent.CUSTOMER.name]
        material_data       = self.db.storage[Ent.MATERIAL.name]
        product_data        = self.db.storage[Ent.PRODUCT.name]

        # Generate the Transportation graph
        transport_graph  = self.transportation_graph if self.transportation_graph else self.create_transportation_graph(as_label)
        if isinstance(transport_graph, tuple):
            transport_graph, _ = transport_graph

        # Get Supply Flow Graph
        supply_flow_graph = self.supply_flow_graph if self.supply_flow_graph else self.create_supply_flow(as_label)

        # Knowledge Graph
        self.knowledge_graph = nx.compose(supply_flow_graph, transport_graph)

        # Add location details
        for entity_type in [material_data, product_data, customer_data, producer_data, supplier_data, distributor_data]:
            for entity in entity_type:

                entity: Material | Product | Customer | Supplier

                # Get Entity name and location
                name = entity.name
                loc  = entity._location

                if loc:
                    # Add label (or title) to link data
                    info = {}
                    if as_label:
                        info['label'] = 'located_at'
                    else:
                        info['title'] = 'located_at'

                    self.knowledge_graph.add_edges_from([(name, loc.name, info)])
        
        return self.knowledge_graph


# Step 1: Convert Raw Data to Standardize Format

## Required Functions

In [16]:
def get_df(sheet_name: str) -> pd.DataFrame:
    return pd.read_excel('Excavator_SupplyChain_Data.xlsx', sheet_name=sheet_name)

In [17]:
# Convert XLXS to CSV
def xlsx_to_csv(sheet_name: str):
    get_df(sheet_name).to_csv(f'{sheet_name}.csv', index=False, header=True)
    print(f'Stored as {sheet_name}.csv')

In [18]:
# Dictionary to Store Nodes
NODES = {
    'locations': {},
    'product': {},
    'supplier': {},
    'producer': {},
    'distributor': {},
    'material': {}
}

DB.reset()
DB.show()

Entity Database


## Transportation Data

### Flatbed

In [19]:
xlsx_to_csv('FlatbedData')
df = pd.read_csv('FlatbedData.csv')
df.info()

Stored as FlatbedData.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2219 entries, 0 to 2218
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   source  2219 non-null   object 
 1   dest    2219 non-null   object 
 2   cost    2219 non-null   float64
 3   time    2219 non-null   float64
dtypes: float64(2), object(2)
memory usage: 69.5+ KB


In [20]:
for fromLoc, toLoc, cost, t in df.iloc[:, :].values:

    # Add location (if not present)
    fromLoc.strip()
    if fromLoc not in NODES['locations']:
        fromLoc = fromLoc.split()[0]
        NODES['locations'][fromLoc] = Location(fromLoc)
    
    toLoc.strip()
    if toLoc not in NODES['locations']:
        toLoc = toLoc.split()[0]
        NODES['locations'][toLoc] = Location(toLoc)
    
    # Add Transportation Cost
    Transporter(NODES['locations'][fromLoc], NODES['locations'][toLoc]).price(cost).time(t)

### Port to Port

In [21]:
xlsx_to_csv('Port-to-Port')
df = pd.read_csv('Port-to-Port.csv')
df.info()

Stored as Port-to-Port.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   source     4500 non-null   object 
 1   dest       4500 non-null   object 
 2   cost_20ft  4500 non-null   float64
 3   time_20ft  4500 non-null   float64
 4   cost_40ft  4500 non-null   float64
 5   time_40ft  4500 non-null   float64
 6   cost_roro  4500 non-null   float64
 7   time_roro  4496 non-null   float64
dtypes: float64(6), object(2)
memory usage: 281.4+ KB


In [22]:
for fromLoc, toLoc, cost_20ft, time_20ft, cost_40ft, time_40ft, cost_roro, time_roro in df.iloc[:, :].values:

    # Add location (if not present)
    fromLoc.strip()
    if fromLoc not in NODES['locations']:
        NODES['locations'][fromLoc] = Location(fromLoc)
    
    toLoc.strip()
    if toLoc not in NODES['locations']:
        NODES['locations'][toLoc] = Location(toLoc)
    
    # Add Transportation Cost
    Transporter(NODES['locations'][fromLoc], NODES['locations'][toLoc], name=f"{fromLoc}==>{toLoc} (20ft)").price(cost_20ft).time(time_20ft)
    Transporter(NODES['locations'][fromLoc], NODES['locations'][toLoc], name=f"{fromLoc}==>{toLoc} (40ft)").price(cost_40ft).time(time_40ft)
    Transporter(NODES['locations'][fromLoc], NODES['locations'][toLoc], name=f"{fromLoc}==>{toLoc} (roro)").price(cost_roro).time(time_roro)

## Node Properties

### Source Production Cost and Capacity

In [23]:
df = get_df("Production_Cost_Capacity")

for source_name, cost, cap in df.iloc[:, :].values:
    
    # Create Producer Node
    NODES['producer'][source_name] = Producer(source_name).at(NODES['locations'][source_name]).varCost(cost).capacity(cap)
    

### Components

In [24]:
df = get_df('ComponentsQuantityAndCube')

for component, qty, length, width, height, cube, cube_per_container, weight, weight_per_container, _, water_transport, land_transport in df.iloc[:, :].values:

    # Extra Properties
    props = {
        'quantity': qty,
        'length': length,
        'width': width,
        'height': height,
        'cube': cube,
        'cube_per_container': cube_per_container,
        'weight': weight,
        'weight_per_container': weight_per_container,
        'water_transport': water_transport,
        'land_transport': land_transport
    }

    # Create Material
    NODES['material'][component] = Material(component).extra_properties(props)

In [25]:
comp_produced_at = {
    'CA': ['source-6', 'source-17', 'source-21'],
    'Linkage': ['source-6', 'source-21'],
    'Axles': ['source-6', 'source-17', 'source-21'],
    'Lights': ['source-6', 'source-17', 'source-21'],
    'Seat': ['source-6', 'source-17', 'source-21'],
    'Radio': ['source-6', 'source-17', 'source-21'],
    'Guard': ['source-6', 'source-17', 'source-21'],
    'Fender': ['source-6', 'source-32', 'source-21'],
    'Bucket': ['source-3', 'source-17', 'source-21'],
    'Tires': ['source-35', 'source-17', 'source-21']
}

for component in comp_produced_at:
    for source in comp_produced_at[component]:
        NODES['material'][component].by(NODES['producer'][source])

In [26]:
DB.show()

Entity Database
	[  1] LOCATION    N1
	[  2] LOCATION    B010
	[  3] LOCATION    B030
	[  4] LOCATION    B150
	[  5] LOCATION    B160
	[  6] LOCATION    B170
	[  7] LOCATION    B190
	[  8] LOCATION    B270
	[  9] LOCATION    B290
	[ 10] LOCATION    B330
	[ 11] LOCATION    B350
	[ 12] LOCATION    B370
	[ 13] LOCATION    B420
	[ 14] LOCATION    D070
	[ 15] LOCATION    D090
	[ 16] LOCATION    D100
	[ 17] LOCATION    D120
	[ 18] LOCATION    D180
	[ 19] LOCATION    D241
	[ 20] LOCATION    D260
	[ 21] LOCATION    D310
	[ 22] LOCATION    D350
	[ 23] LOCATION    D390
	[ 24] LOCATION    D420
	[ 25] LOCATION    D430
	[ 26] LOCATION    D440
	[ 27] LOCATION    D470
	[ 28] LOCATION    D480
	[ 29] LOCATION    D500
	[ 30] LOCATION    D600
	[ 31] LOCATION    E070
	[ 32] LOCATION    E130
	[ 33] LOCATION    E140
	[ 34] LOCATION    E250
	[ 35] LOCATION    E300
	[ 36] LOCATION    E30A
	[ 37] LOCATION    E330
	[ 38] LOCATION    E459
	[ 39] LOCATION    E480
	[ 40] LOCATION    E490
	[ 41] LOCATION    E500
	[

# Step 2: Creating Graph

In [27]:
net = SupplyNetwork()
net.add_DB(DB)

In [28]:
# Transportation Graph
transportation_graph = net.create_knowledge_graph(as_label=False)

In [ ]:
# Supply Flow
supply_flow = net.create_supply_flow(as_label=False)

In [ ]:
# Knowledge Graph
knowledge_graph = net.create_knowledge_graph(as_label=False)

# Step 3: Analysis

- **Transportation Graph:** Only Transportation Data

- **Supply Flow Graph:** All Data, except transportation
- **Knowledge Graph:** All Data, complete graph